In [ ]:
# 3. STEP
# In this notebook, the names from 'names.csv' are classified by gender using the NamSor API.
# The results are saved in 'names_cat.csv'

# https://www.namsor.com/
# https://v2.namsor.com/NamSorAPIv2/apidoc.html
# using NamSor API v2 Python SDK
# https://github.com/namsor/namsor-python-sdk2
# licensed under GNU Affero General Public License v3.0

In [ ]:
# >>> Import 'names.csv'

import pandas

print("Importing names... ")
names = pandas.read_csv("data/names.csv", usecols=["name", "n_publs", "likely_gender", "score"])

# Setting index & accessing cells: https://pythonhow.com/accessing-dataframe-columns-rows-and-cells/
names = names.set_index("name", drop = False)
print("Names imported. They look like this: {}".format(names[:10]))

In [ ]:
# >>> Preparing for API use

# Get private API Key for NamSor API v2 (contained in txt file)
print("Getting private key... ")
key = ''

# Import personal key
with open("key.txt", "r") as file:
    key = file.read()

if(len(key) > 0):
    print("Got private key.")
else: 
    print("Could not find private key. Please check the file name and make sure you have an API key.")

In [ ]:
# Following scripts partly taken from https://github.com/namsor/namsor-python-sdk2 "Getting Started" 
# and adapted to keep key private and remove unnecessary lines.

print("Setting up NamSor API v2 connection settings...")

import openapi_client
from openapi_client.rest import ApiException

# Configure API key authorization: api_key
configuration = openapi_client.Configuration()
configuration.api_key['X-API-KEY'] = key

# create an instance of the personal API class
pers_api_instance = openapi_client.PersonalApi(openapi_client.ApiClient(configuration))

In [ ]:
# Formatting the names using the API's models
def format_full_name(name):
    "This function formats a full name. It takes a full name and returns a PersonalNameIn"
    return openapi_client.PersonalNameIn(id=name, name=name)

In [ ]:
def format_full_names(li):
    "This function formats multiple full names. It takes a list of unformatted full names and returns a list of formatted full names."
    return list(map(format_full_name, li))

In [ ]:
def format_full_name_batch(li):
    "This function formats a batch of formatted full names. It takes a list of formatted full names and returns a formatted batch."
    return openapi_client.BatchPersonalNameIn(personal_names=li)

In [ ]:
def fullname_batch(batch):
    "This function calls the API. It takes a formatted batch of full names and returns the API response."
    return pers_api_instance.gender_full_batch(batch_personal_name_in=batch)

In [ ]:
def call_api_fullname_batch(li):
    "This function prepares a list of unformatted names for the API call and then calls the API calling function. It returns the API's name classifications."
    current_batch = format_full_names(li)  # format the names
    batch_personal_name_in = format_full_name_batch(current_batch)# format the batch
    api_response =  fullname_batch(batch_personal_name_in)# call api
    return api_response.personal_names # return result

In [ ]:
# >>> Classifying names with API

# Sending in one batch at a time and saving the result answer by answer.

batch_size = 1000 #1000 is the API limit given by NamSor
start = 0
end = batch_size
result = []

names_stack = list(names.index.values)

print('Will need to make {} calls.'.format(len(names_stack) / batch_size))

In [ ]:
while (len(names_stack) >= batch_size):
    try:
        result = result + call_api_fullname_batch(names_stack[start:end])
        del names_stack[start:end] # delete the names that have already been categorized from the stack
        print("Batch of names analyzed. {} names left.".format(len(names_stack)))
        
        # categorize remaining names if they are less than a batch size
        if(len(names_stack) < batch_size and len(names_stack) > 0):
            result = result + call_api_fullname_batch(names_stack)
            names_stack = [] # empty the stack
            print("Batch of names analyzed. {} names left.".format(len(names_stack)))
    except ApiException as e: # Sometimes with a big batch of batches, the API calling gets interrupted (don't panic!)
        print("Exception when calling PersonalApi: gender_full_batch: {}".(e))
        if((len(list(names.index.values))-len(result)) == len(names_stack)): #check that no names got lost
            print("No names got lost. Trying again with stack size {}...".format(len(names_stack)))
            continue

if(len(names_stack) != 0):
    try:
        result = result + call_api_fullname_batch(names_stack)
        names_stack = []
        print("Batch of names analyzed. {} names left.".format(len(names_stack)))
    except ApiException as e:
        print("Exception when calling PersonalApi: gender_full_batch: {}".format(e))

print("All batches analyzed.")

In [ ]:
# >>> Save results to dataframe

# Convert results (list of openapi_client.models.personal_name_gendered_out.PersonalNameGenderedOut) to (list of dictionaries)
print('Filling the results into the names dataframe...')
for oapi_el in result:
    names.at[oapi_el.id, 'likely_gender'] = oapi_el.likely_gender
    names.at[oapi_el.id, 'score'] = oapi_el.score
print('Dataframe completed with API results. Here are some results: {}'.format(names[:10]))

In [ ]:
# Correction by hand of two names that immediately stood out:
print('Correcting two names...')

names.at['(David) Jing Dai', 'likely_gender'] = 'male'
names.at['(David) Jing Dai', 'score'] = 7

names.at['(Sophy) Shu-Jiun Chen', 'likely_gender'] = 'female'
names.at['(Sophy) Shu-Jiun Chen', 'score'] = 7

print('Corrected! See {}'.format(names[:5]))

In [ ]:
# Saving results to 'names_cat.csv'
print("Saving categorized names to CSV... ")
names.to_csv("data/names_cat.csv")
print("Names saved!")